<a href="https://colab.research.google.com/github/Tessellate-Imaging/monk_v1/blob/master/study_roadmaps/4_image_classification_zoo/Classifier%20-%20Plant%20Disease%20Zindi%20Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Monk Library - https://monkai.org/


## Competition - https://zindi.africa/competitions/iclr-workshop-challenge-1-cgiar-computer-vision-for-crop-disease

# Table of contents


## Install Monk


## Using pretrained model for classifying plat diseases in images


## Training a classifier from scratch

# Installation

In [ ]:
! git clone https://github.com/Tessellate-Imaging/monk_v1.git

In [ ]:
# If using Colab install using the commands below
!cd monk_v1/installation/Misc && pip install -r requirements_colab.txt

# If using Kaggle uncomment the following command
#!cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

# Select the requirements file as per OS and CUDA version when using a local system or cloud
#!cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt

# Used trained classifier for demo

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Download trained weights

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Vrfo64xm_fRpow1uQbzntpoAR2HAaCPr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Vrfo64xm_fRpow1uQbzntpoAR2HAaCPr" -O cls_plantzindi_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq cls_plantzindi_trained.zip 

In [ ]:
ls workspace/Project-Plant-Zindi

In [ ]:
# Gluon project
from gluon_prototype import prototype

In [ ]:
# Load project in inference mode

gtf = prototype(verbose=1);
gtf.Prototype("Project-Plant-Zindi", "Gluon-densenet161", eval_infer=True);

#Other trained models - uncomment 
#gtf.Prototype("Project-Plant-Zindi", "Gluon-densenet169", eval_infer=True);
#gtf.Prototype("Project-Plant-Zindi", "Gluon-densenet201", eval_infer=True);

In [ ]:
#Infer

In [ ]:
img_name = "workspace/test/1.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/2.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/3.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

# Training custom classifier from scratch

## Data Download
- Its the same dataset, with all files converted to jpg format

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1CZ640Qq6RQk8TSTDQgt2c0xmk-_Enh_0' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1CZ640Qq6RQk8TSTDQgt2c0xmk-_Enh_0" -O data.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq data.zip

# Training

In [ ]:
# Find documentation here - https://clever-noyce-f9d43f.netlify.com/#/introduction
# Find example here - https://github.com/Tessellate-Imaging/monk_v1/tree/master/examples

In [ ]:
import sys
sys.path.append("monk_v1/monk/")

In [ ]:
# Using gluoncv backend, you can use pytorch or keras too. 
# Check https://github.com/Tessellate-Imaging/monk_v1/tree/master/examples

In [ ]:
from gluon_prototype import prototype

In [ ]:
# Step 1 - Create experiment
ptf = prototype(verbose=1);
ptf.Prototype("Project-1", "Experiment-2");

In [ ]:
# Step 2 - Invoke Quick Prototype Default mode
ptf.Default(dataset_path="data/train",
           model_name="densenet201", freeze_base_network=True,
           num_epochs=20);

In [ ]:
ptf.update_batch_size(8);
ptf.update_trainval_split(0.85);
ptf.update_freeze_base_network(False);
ptf.update_learning_rate(0.001);

In [ ]:
ptf.Reload();

In [ ]:
ptf.Train();

# Inference

In [ ]:
# Make sure the workspace directory exists in the same path this notebook

In [ ]:
import sys
sys.path.append("monk_v1/monk/")

In [ ]:
from gluon_prototype import prototype

In [ ]:
ptf = prototype(verbose=1);
ptf.Prototype("Project-1", "Experiment-2", eval_infer=True);

In [ ]:
inference_dataset = "data/test/";
output = ptf.Infer(img_dir=inference_dataset, return_raw=True);

In [ ]:
output[3]

In [ ]:
#scores_order = ["healthy_wheat", "leaf_rust", "stem_rust"]

In [ ]:
from scipy.special import softmax

In [ ]:
combined = [];
columns = ["ID", "leaf_rust", "stem_rust", "healthy_wheat"];

for i in range(len(output)):
    img_id = output[i]["img_name"].split(".")[0];
    scores = output[i]["raw"];
    probs = softmax(scores);
    combined.append([img_id, probs[1], probs[2], probs[0]]);

In [ ]:
import pandas as pd
df = pd.DataFrame(combined, columns = columns)  

In [ ]:
df.to_csv("submission.csv", index=False)